## TRT Experiments:

For each experiment summarized below, a table accompanies the stated hypothesis which describes the dataset(s) used to test the hypothesis. In each column, 'All' indicates that the comparison is made along the specified variable, a '1' indicates a setting in which the test is performed multiple times using a single value of this variable for each test, and 'Ref' refers to using the single reference execution. A value of 'Indep.' refers to tests which are performed for each single value independently, and 'Indep. & Compare' refers to variables for which a two-sample test comparing each value of this variable will be performed following independent evaluation. To avoid false-positive inflation due to multiple comparisons, the mean and standard deviation statistic and p-value for each test will be reported.

In total, following this structure, 9 tests will be performed for each of two pipelines and two perturbation modes, resulting in 36 one-sample tests. For each of these 9 tests, 2 two-sample tests will be performed, comparing pipelines for each perturbation mode, resulting in an additional 18 tests, bringing the total number of tests to 54.

### Hypothesis 1: Individual Variation

$H_0$: Individuals are not more different from one another than they are from themselves<br>
$H_A$: Individuals are distinct across {sessions, directions, simulations}<br>
Expected result: Accept $H_A$

Collections:

|  #  | Participants | Sessions | Directions | Simulations | Pipelines        | Perturbation     |
|:---:|:-------------|:---------|:-----------|:------------|:-----------------|:-----------------|
| 1.1 | All          | All      | 1          | All         | Indep. & Compare | Indep.           |
| 1.2 | All          | All      | 1          | Ref         | Indep. & Compare | Indep.           |
| 1.3 | All          | 1        | All        | All         | Indep. & Compare | Indep.           |
| 1.4 | All          | 1        | All        | Ref         | Indep. & Compare | Indep.           |
| 1.5 | All          | 1        | 1          | All         | Indep. & Compare | Indep.           |


### Hypothesis 2: Session Variation

$H_0$: Sessions are not different from one another<br>
$H_A$: Sessions are distinct from one another across {directions, simulations}<br>
Expected result: Fail to reject $H_0$

Collections:

|  #  | Participants | Sessions | Directions | Simulations | Pipelines        | Perturbation     |
|:---:|:-------------|:---------|:-----------|:------------|:-----------------|:-----------------|
| 2.6 | 1            | All      | All        | All         | Indep. & Compare | Indep.           |
| 2.7 | 1            | All      | All        | Ref         | Indep. & Compare | Indep.           |
| 2.8 | 1            | All      | 1          | All         | Indep. & Compare | Indep.           |


### Hypothesis 3: Direction Variation

$H_0$: Odd- and Even-direction sets are not different from one another<br>
$H_A$: Direction subsets are distinct from one another<br>
Expected result: Fail to reject $H_0$

Collections:

|  #  | Participants | Sessions | Directions | Simulations | Pipelines        | Perturbation     |
|:---:|:-------------|:---------|:-----------|:------------|:-----------------|:-----------------|
| 3.9 | 1            | 1        | All        | All         | Indep. & Compare | Indep.           |


In [1]:
import pandas as pd
import numpy as np
import os.path as op
import os

from hyppo import discrim
# from varmodel_utils import objective, get_full_variance_indices,\
#                            get_sub_variance_indices, prune

In [2]:
bp = '/data/RocklandSample/derivatives/paper1/'
exp = 'figures'
bpp = op.join(bp, exp)
try:
    os.makedirs(bpp)
except FileExistsError:
    pass

# cols = ["subject", "session", "directions", "pipeline", "simulation"]

df_pyonly = pd.read_hdf(bp + 'connectomes_mp_pyonly.h5')
df_pyonly['Instrumentation'] = "Inputs"
# df_pyonly = df_pyonly.reset_index(level=0).set_index(cols)

df_fs = pd.read_hdf(bp + 'connectomes_mp.h5')
df_fs['Instrumentation'] = "Pipeline"
# df_fs = df_fs.reset_index(level=0).set_index(cols)

# df = pd.concat([df_fs, df_pyonly], axis=0, ignore_index=True)

In [68]:
stat_cols = ['hypothesis', 'test', 'pipeline', 'instrumentation', 'discrim', 'p-value']

def pipeline_discrims(df_, idvar='subject', hyp=0, test=0, instrum='NA', reps=100, workers=4):
    stat_list = []
    xs = []
    ys = []
    for pipe in ['det', 'prob']:
#         print(pipe)
        df = df_[df_['pipeline'] == pipe]
        x = np.array([np.reshape(g, -1) for g in df.graph.values])
        xs += [x]

        # Determine membership 
        uniq = list(df[idvar].unique())
        y = np.array([uniq.index(s) for s in df[idvar]])
        ys += [y]

        disc, pval = discrim.DiscrimOneSample().test(x, y, reps=reps, workers=workers)
#         print(disc, pval)
        tmpstat = {stat_cols[0]: hyp,
                   stat_cols[1]: test,
                   stat_cols[2]: pipe,
                   stat_cols[3]: instrum,
                   stat_cols[4]: disc,
                   stat_cols[5]: pval}

        stat_list += [tmpstat]
        del tmpstat

    assert(np.equal(ys[0], ys[1]).all())
    d1, d2, pval = discrim.DiscrimTwoSample().test(xs[0], xs[1], y, reps=reps, workers=workers)
#     print(d1, d2, pval)
    tmpstat = {stat_cols[0]: hyp,
               stat_cols[1]: test,
               stat_cols[2]: 'det-prob',
               stat_cols[3]: instrum,
               stat_cols[4]: d1-d2,
               stat_cols[5]: pval}
    stat_list += [tmpstat]
    del tmpstat

    return stat_list

In [59]:
stat_list = []
reps = 3
workers = 4
np.random.seed(42)

ssets = [[(s, df_[df_['subject']==s]['session'].unique()[i])
          for s in df_['subject'].unique()]
         for i in range(2)]

for df_ in [df_fs, df_pyonly]:
    instrum = df_['Instrumentation'].values[0]
    print(instrum)

    ## Hypothesis 1: cross-subject
    hyp=1

    ### Tests 1 & 2
    for dir_ in df_['directions'].unique():
        ### Test 1: Multi-session, 1 dir, all sims
        test=1
        df__ = df_[df_['directions'] == dir_]
        stat_list += pipeline_discrims(df__, hyp=hyp, test=test, instrum=instrum, reps=reps)

        ### Test 2: Multi-session, 1 dir, reference execution
        test=2
        df__ = df__[df__['simulation'] == 'ref']
        stat_list += pipeline_discrims(df__, hyp=hyp, test=test, instrum=instrum, reps=reps)

        del df__
        
    ### Tests 3, 4 & 5
    for subses in ssets:
        ### Test 3: Multi-direction, 1 session, all sims
        test=3
        qbase = '(subject == "{0}" and session == "{1}")'
        q = [qbase.format(s[0], s[1]) for s in sets[0]]
        df__ = pd.concat([df_.query(q_) for q_ in q])
        stat_list += pipeline_discrims(df__, hyp=1, test=test, instrum=instrum, reps=reps)

        ### Test 4: Multi-direction, 1 session, reference execution
        test=4
        df_1 = df__[df__['simulation'] == 'ref']
        stat_list += pipeline_discrims(df_1, hyp=1, test=test, instrum=instrum, reps=reps)

        ### Test 5: Multi-simulation, 1 dir, 1 session
        test=5
        for dir_ in df__['directions'].unique():
            df_2 = df__[df__['directions'] == dir_]
            stat_list += pipeline_discrims(df_2, hyp=1, test=test, instrum=instrum, reps=reps)

        del df__, df_1, df_2

Pipeline
det


/Users/greg/code/env/stability/lib/python3.7/site-packages/hyppo/_utils.py:67: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)


0.8158580369817303 0.3333333333333333
prob


/Users/greg/code/env/stability/lib/python3.7/site-packages/hyppo/_utils.py:67: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)


0.8196443089430895 0.3333333333333333


...

0.9080971277399849 0.3333333333333333
prob


/Users/greg/code/env/stability/lib/python3.7/site-packages/hyppo/_utils.py:67: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)


0.888212396069539 0.3333333333333333


/Users/greg/code/env/stability/lib/python3.7/site-packages/hyppo/_utils.py:67: RuntimeWarning: The number of replications is low (under 1000), and p-value calculations may be unreliable. Use the p-value result, with caution!
  warnings.warn(msg, RuntimeWarning)


0.9080971277399849 0.888212396069539 0.3333333333333333


In [67]:
pd.DataFrame.from_dict(stat_list)[0:50]

,hypothesis,test,pipeline,instrumentation,discrim,p-value
0,1,1,det,Pipeline,0.815858,0.333333
1,1,1,prob,Pipeline,0.819644,0.333333
2,1,1,det-prob,Pipeline,-0.003786,1.000000
3,1,2,det,Pipeline,0.640000,0.333333
4,1,2,prob,Pipeline,0.647917,0.333333
5,1,2,det-prob,Pipeline,-0.007917,1.000000
6,1,1,det,Pipeline,0.815789,0.333333
7,1,1,prob,Pipeline,0.816677,0.333333
8,1,1,det-prob,Pipeline,-0.000889,1.000000
9,1,2,det,Pipeline,0.640833,0.333333
